# 1. Introduction
Evaluation of derivatives is integral to many machine learning methods. For this purpose, two main methods could be used: symbolical and numerical differentiation. Symbolical differentiation, though straightforward, its implementation requires complex expression manipulation in computer algebra systems, making it very costly to evaluate; the method is also limited to closed-form input expressions. On the other hand, numerical differentiation computes the function derivative by approximating it using small values of step size h; though numerical simpler and faster than symbolic diffferentiation, it suffers from stability issues and round-off or truncation errors.

To address the weaknesses of both these methods, automatic differentiation (AD) was introduced. Since, it has been applied in different areas, such as  engineering design optimization, structural mechanics, and atmospheric sciences; its application to machine learning methods popularised AD. Therefore, due to the important role AD plays in many scientific fields, we introduce a python package that provides user-friendly methods for performing forward-mode AD. Our package supports the evaluation of first derivatives of functions defined by user at given input value. 

# 2. Background
All numerical computation can be seen as a combination of elementary operations for which the derivatives are known. The derivatives of the overall composition can be found by combining the derivatives of elementary operations through the chain rule. Such elementary functions include arithmetic operations (addition, subtraction, multiplication, division), sign switch, and functions such as exponential, logarithm, and the trigonometric (e.g. sin(x), cos(x)). Traces of these elementary operations can be represented by a trace table or a computational graph. Trace table are originally used to trace the value of variables as each line of code is executed. As an example of this flow, Table 1 shows the evaluation trace of elementary operations of the computation f(x<sub>1</sub>) = ln(x<sub>1</sub>) + 3 * x<sub>1</sub> and Figure 1 gives an example of a graphic representation of function f(x<sub>1</sub>) by its elementary operations. 

| Trace       | Elementary function | Current Function Value | Function Derivative |
| ------------- |:-------------:|:-------------:|:-------------:|
| X1      | X1            |  c             | 1  |
| X2      | ln(X1)            | ln(c)      | 1/c |
| X3      | 3 * X1            |  3c        | 3   |
| X4      | X2 + X3             | ln(c) + 3c | 1/c + 3 |

![alt text](sample_trace_graph.png "Figure 1")

The forward mode of AD starts from the input value and compute the derivative of intermediate variables with respect to the input value. Applying the chain rule to each elementary operation in the forward primal trace, we generate the corresponding derivative trace, which gives us the derivative in the final variable. Forward  mode AD can also be viewed as evaluating a function using dual numbers, which are defined as a+b*&epsilon;, where a, b &#1013; R and &epsilon; is a nilpotent number such that &epsilon;^2 = 0 and &epsilon; &ne; 0. It can be shown that the coefficient of &epsilon; after evaluating a function is exactly the derivative of that function, which also works for chain rule.

# 3. Installation
Th package uses a setup.py file for a simple installation, you need to to run the following command in the package main folder:

```
pip install .
```

# 4. How to Use AutoDiff 
In this section, we demo the the AutoDiff package and its capabilities:

In [1]:
import AutoDiff as ad

The AutoDiff package can be used in two different ways:

### 4.1. Pass the function to AD():
You can pass disrectly a \[function, a variable label, and variable value\] to AD as follows:

```
AD(function: str, variable_label:str, variable_value: float): -> (value, derivative)
```

In [18]:
## y = f(x)
y = ad.AD('-x**3 + 2*x**2 + 3*x + 5', 'x', 2)
print("val: ", y.val,"\nder: ", y.der)
print(y)

val:  11 
der:  -1
AD Object: Value = 11.000, Derivative =-1.000


In [17]:
## z = f(w)
z = ad.AD('-w**3 + 2*w**2 + 3*w + 5', 'w', 2)
print("val: ", z.val,"\nder: ", z.der)
print(z)

val:  11 
der:  -1
AD Object: Value = 11.000, Derivative =-1.000


### 4.2. Create an AD_Object():
Alternatively, you can delcare an AD_Object(value) and use it as a variable; the AD object will store the value and derivative with each subsequent elementary operation.  

In [14]:
x = ad.AD_Object(2)
y = -x**3 + 2*x**2 + 3*x + 5
print("val: ", y.val,"\nder: ", y.der)
print(y)

val:  11 
der:  -1
AD Object: Value = 11.000, Derivative =-1.000


In [16]:
w = ad.AD_Object(2)
z = -w**3 + 2*w**2 + 3*w + 5
print("val: ", z.val,"\nder: ", z.der)
print(z)

val:  11 
der:  -1
AD Object: Value = 11.000, Derivative =-1.000


### 4.3. Supported Elementary Functions:
    

The following elementary functions are currently supported by the AutoDiff package:
- addition[+], subtraction[-], multiplication[*] and division[/]
- power [can be called by pow() or using **]
- sine [sin], cosine [cos], tangent [tan]
- natural log [ln], exponential [**note: exp should be called by e() and not exp()**]

We will add more functions as the project proceeds; we envision having most functions implemented in the math module of the standard python library.  

# 5. Software Organization
The package has the following directory structure:
- README.md
- LICENSE.md
- setup.py
- requirements.txt
- test/ (using CodeCov and TravisCI)
    - \__init\__.py
    - test_AutoDiff.py
- docs
    - milestone1.md
    - milestone2.ipynb
    - sample_trace_graph.png
- AutoDiff/
    - \__init\__.py
    - AutoDiff_Class.py

The package uses standard .py standalone files and setup.py for package installation; it will be also distributed through
conda. Finallt, we implemented 31 tests in the test directory; the pytest framework is used, so the tests can be simply run by invoking pytest in that folder. TravisCI and CodeCov are used for integeration and coverage.

# 6. Implementation
We implemented an Automatic Differentiation Class (*AD*) that stores and tracks the value of the given function and its gradient. This class calls another class, AD_Object(), which intitializes the given variable of the function as an automatic differentiation object.  

Main AD class attributes are:
1. a: initial variable value for evaluation of the function and gradient.
2. var_label: variable label to be used (x, y, z, ...etc), for which the derivative of the function is to be returned 
2. function_value: the function value at that variable_values. 
3. derivative_values: the function gradient, derivatives wrt to each declared variable at variable_values.

The *AD* class implements all relevant dunder methods(\__repr\__, \__add\__, \__mul\__, etc.). In addition, we will implement a “summing_everything_up” function that takes as an input the function and the values at which we want to compute the gradient. This function will create the neural network of elementary operations for the input function and then compute the grad of the composition of those elementary operations that constitute the function, using the chain rule.

For the elementary functions, we will implement all relevent functions from the *math* module of the Python Standard Library, such as (exp, sin, cos, sin, etc.). The functions will be able to handle two types of input: AD objects and numbers. For the first input type , the elem_function will return the function evaluation and derivative; for the second type, it will behave as a math.function and just return the evaluation.

Finally, the package will be designed to have minimal dependencies; we envision only the *numpy* and *math* packages will be needed.